In [1]:
import pandas as pd
import numpy as np
#import limetr
import matplotlib.pyplot as plt
import sys
path = '../code/'
sys.path.insert(0,path)
from lme_forecast import LME
import copy

In [2]:
data = pd.read_csv('./data/tb_prevalence.csv')
data = data.sort_values(by=['location_id','age_group_id','sex_id','year_id'])

#### The model
$$ \log(y) = \beta_I\text{age-sex} + \beta_1 \text{HAQ} + \pi_{l} $$
$$ \log(y) = \beta_I\text{age-sex} + \beta_1 (\text{HAQ} - \overline{\text{HAQ}}) + \pi_{l} $$

In [4]:
lme_fit_no_random = data['lme_fit'].values

In [3]:
X = np.zeros((data['value'].values.shape[0],2))
X[:,0] = data['haq'].values
X[:,1] = data['haq'].values - np.mean(data['haq'].values)

- use HAQ directly

In [5]:
model = LME(data['value'].values,X,[0],ran_intercept=True)
import time
t0 = time.time()
model.optimize(inner_max_iter=1000,outer_max_iter=1,share_obs_std=True)
print('elapsed',time.time()-t0)

number of datapoints 251160
number of studies 195
number of global effects coefs 47
number of random effects coefs 1
total number of fixed effects variables 49
fit with gamma fixed...
finished...elapsed 2.168102979660034
elapsed 23.071925163269043


- use centered HAQ

In [6]:
model2 = LME(data['value'].values,X,[1],ran_intercept=True)
import time
t0 = time.time()
model2.optimize(inner_max_iter=1000,outer_max_iter=1,share_obs_std=True)
print('elapsed',time.time()-t0)

number of datapoints 251160
number of studies 195
number of global effects coefs 47
number of random effects coefs 1
total number of fixed effects variables 49
fit with gamma fixed...
finished...elapsed 1.9841079711914062
elapsed 3.486696243286133


- compare y fit from two models

In [7]:
print(model.gamma_soln, model.delta_soln)
print(model2.gamma_soln, model2.delta_soln)
print(np.linalg.norm(model.yfit - model2.yfit))
print(np.linalg.norm(model.yfit_no_random-model2.yfit_no_random))

[0.15209316] [0.02111712]
[0.15209213] [0.02111712]
7.320150688897461e-06
0.006619792467840186


- compare with lme4 fit

In [8]:
print(np.linalg.norm(model.yfit_no_random- lme_fit_no_random))
print(np.linalg.norm(model2.yfit_no_random- lme_fit_no_random))

0.09505403928195257
0.0898879680881674
